## 자전거 대여 수요 예측

### Project Overview
- 난이도 : *
- 경진대회명 : 자전거 대여 수요 예측 경진대회
- 목표 : 날짜, 계절, 근무일 여부, 날씨, 온도, 체감 온도, 풍속 데이터를 활용하여 자전거 대여 수량 예측
- 문제 유형 : 회귀
- 평가지표 : RMSLE
- 제출 시 사용 모델 : Random Forest Regression
- 사용 라이브러리 및 버전
    - numpy==1.19.5
    - pandas==1.3.2
    - seaborn==0.11.2
    - matplotlib==3.4.3
    - scikit-learn==0.23.2
    - datetime, calendar

### 학습 목표
- 머신러닝 모델링 프로세스와 기본적인 회귀 모델을 배움